<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/StockAndNewsAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

!pip install pandas-datareader

In [0]:
import urllib
import json
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime, date


In [0]:
def get_data(symbol, start, end):
  # Use IEX API for stats so we get all info we need  IEX Cloud consol 
  https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=symbol,companyName,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent
  # From here we can get the following information
  """
  symbol	string	
  companyName	string	Company name of the security
  marketcap	number	Market cap of the security calculated as shares outstanding * previous day close.
  week52high	number	
  week52low	number	
  week52change	number	Percentage change
  sharesOutstanding	number	Number of shares outstanding as the difference between issued shares and treasury shares. Investopedia
  avg30Volume	number	Average 30 day volume
  avg10Volume	number	Average 10 day volume
  float	number	Returns the annual shares outstanding minus closely held shares.
  employees	number	
  ttmEPS	number	Trailing twelve month earnings per share. Investopedia
  ttmDividendRate	number	Trailing twelve month dividend rate per share
  dividendYield	number	The ratio of trailing twelve month dividend compared to the previous day close price. The dividend yield is represented as a percentage calculated as (ttmDividendRate) / (previous day close price) Investopedia
  nextDividendDate	string	Expected ex date of the next dividend
  exDividendDate	string	Ex date of the last dividend
  nextEarningsDate	string	Expected next earnings report date
  peRatio	number	Price to earnings ratio calculated as (previous day close price) / (ttmEPS)
  beta	number	Beta is a measure used in fundamental analysis to determine the volatility of an asset or portfolio in relation to the overall market. Levered beta calculated with 1 year historical data and compared to SPY.
  day200MovingAvg	number	
  day50MovingAvg	number	
  maxChangePercent	number	
  year5ChangePercent	number	
  year2ChangePercent	number	
  year1ChangePercent	number	
  ytdChangePercent	number	
  month6ChangePercent	number	
  month3ChangePercent	number	
  month1ChangePercent	number	
  day30ChangePercent	number	
  day5ChangePercent	number	
  
  
  """
  
  
  data =  dr.DataReader(symbol, 'iex', start, end)[['close']]
  return data.rename(columns={'close': symbol})

SyntaxError: ignored

<h2>Authenticate User </h2>

In [0]:
from google.colab import auth
auth.authenticate_user()

<h3> Loading Nasdaq and Nyse shares </h3>

In [0]:
!gsutil cp gs://datascience-bucket-mm/nyse-companylist.csv /tmp/nyse.csv
!gsutil cp gs://datascience-bucket-mm/nasdaq-companylist.csv /tmp/nyse.csv  
  
# Print the result to make sure the transfer worked.
#!cat /tmp/nyse.csv

Copying gs://datascience-bucket-mm/nyse-companylist.csv...
/ [1 files][392.3 KiB/392.3 KiB]                                                
Operation completed over 1 objects/392.3 KiB.                                    
Copying gs://datascience-bucket-mm/nasdaq-companylist.csv...
/ [1 files][462.5 KiB/462.5 KiB]                                                
Operation completed over 1 objects/462.5 KiB.                                    


<h3>  Testing Historical API </h3>

In [0]:

def get_statistics(ticker):
  base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol=ticker)
  df = pd.read_csv(base_url)
  df['Symbol'] = ticker
  return df

def get_historical_data(ticker, start, end):
  df = get_statistics(ticker)
  return df
  
  

def get_date_ranges():
  end_date = date.today()
  start_date = end_date - BDay(60)
  return start_date, end_date

def test():
  start,end = get_date_ranges()
  print('start:{}, end:{}'.format(start_date, end_date))
  print(get_historical_data('AMZN', start_date, end_date))
  
def get_latest_price(symbol):
  base_url = "https://cloud.iexapis.com/stable/stock/{ticker}/quote?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=symbol,close".format(ticker=symbol)
  import requests
  return pd.read_csv(base_url)

def get_latest_price_yahoo(symbol, as_of_date):
  return dr.get_data_yahoo(symbol, as_of_date, as_of_date)[['Close']]
  
    
  

<h3> Reading source data and computing performance </h3>

In [0]:
def get_nyse_df():
  return pd.read_csv('/tmp/nyse.csv', header=0)[['Symbol', 'Name', 'Sector', 'industry']]


In [0]:
def compute_performance(start_dt, end_dt, ticker):
  try:
    import time
    historical_df =  get_historical_data(ticker, start_dt, end_dt)
    latest_df = get_latest_price(ticker)
    merged = pd.merge(historical_df, latest_df, how='inner' , left_on="Symbol", right_on="symbol").drop('symbol', axis=1)
    return merged
  except Exception as e:
    print('Exception:{}'.format(str(e)))
    print('Unable to find data for {}:{}'.format(ticker,str(e)))

In [0]:
def find_best_performing(start_dt, end_dt):
  print('Finding Best Performing Stocks between:{}-{}'.format(start_dt, end_dt))
  nyse_df = get_nyse_df()
  symbols = nyse_df['Symbol'].values.tolist()
  print('Now we have to source data for:{}'.format(len(symbols)))
  dfs = (compute_performance(start_dt, end_dt, symbol) for symbol in symbols)
  filtered = (df for df in dfs if df is not None)
  all_data = pd.concat(filtered)
  return pd.merge(nyse_df, all_data, how='inner', on='Symbol' )[['Symbol', 'Name', 'Sector', 'industry', 'companyName','close', 
       'month1ChangePercent','month3ChangePercent', 'month6ChangePercent',  'day200MovingAvg', 'day50MovingAvg']]
  

start_dt, end_dt = get_date_ranges()
perf_df = find_best_performing(start_dt, end_dt)#compute_performance(start_dt, end_dt)
print(perf_df.columns)
print(perf_df.shape)

Finding Best Performing Stocks between:2019-07-01 00:00:00-2019-09-22


FileNotFoundError: ignored

In [0]:
# Sorting 
perf_df.sort_values(by=['month1ChangePercent'], inplace=True, ascending=False)
perf_df.head(30)


,Symbol,Name,Sector,industry,companyName,close,month1ChangePercent,month3ChangePercent,month6ChangePercent,day200MovingAvg,day50MovingAvg
690,CTXRW,"Citius Pharmaceuticals, Inc.",Health Care,Major Pharmaceuticals,"Citius Pharmaceuticals, Inc.",0.3000,3.285714,0.578947,-0.285714,0.2786,0.1174
301,ASV,"ASV Holdings, Inc.",Capital Goods,Construction/Ag Equipment/Trucks,"ASV Holdings, Inc.",6.9600,2.962571,1.773800,1.208439,3.2500,3.7700
181,AMRHW,"Ameri Holdings, Inc.",Technology,Semiconductors,"Ameri Holdings, Inc.",0.0900,2.000000,2.000000,-0.400000,0.1657,0.0496
2466,SKIS,"Peak Resorts, Inc.",Consumer Services,Hotels/Resorts,"Peak Resorts, Inc.",10.8400,1.844156,1.433333,1.319031,4.7300,4.5600
3012,TRNX,"Taronis Technologies, Inc.",Technology,Industrial Machinery/Components,"Taronis Technologies, Inc.",0.5180,1.347347,-0.185764,-0.879744,2.4932,0.3582
1602,IFMK,iFresh Inc.,Consumer Services,Food Chains,"iFresh, Inc.",1.9400,1.166667,0.962264,0.507246,1.2400,1.2300
1063,EVOK,"Evoke Pharma, Inc.",Health Care,Major Pharmaceuticals,"Evoke Pharma, Inc.",1.1400,1.050847,0.689944,-0.572438,1.7800,0.7100
31,ACST,"Acasti Pharma, Inc.",Health Care,Major Pharmaceuticals,"Acasti Pharma, Inc.",2.3400,1.009901,1.255556,0.970874,0.9900,1.1000
1081,EYEGW,"Eyegate Pharmaceuticals, Inc.",Health Care,Major Pharmaceuticals,"EyeGate Pharmaceuticals, Inc.",0.0290,1.000000,-0.200000,-0.555556,0.0683,0.0400
604,CETXW,Cemtrex Inc.,Capital Goods,Industrial Machinery/Components,"Cemtrex, Inc.",0.0400,1.000000,-0.800000,-0.800000,0.1548,0.0276


<h3> Group by sector, to find best performers </h3>

In [0]:
res = perf_df[['industry', 'month1ChangePercent','month3ChangePercent', ]].groupby(['industry']).mean().sort_values(by=['month1ChangePercent','month1ChangePercent'], ascending=False)
res.head(20)

,month1ChangePercent,month3ChangePercent
industry,,
Construction/Ag Equipment/Trucks,0.623505,0.391324
Food Chains,0.224757,0.162845
Oil/Gas Transmission,0.196226,0.054032
Shoe Manufacturing,0.180125,-0.129228
Other Pharmaceuticals,0.177430,0.163765
Medical Electronics,0.145786,-0.042202
Aluminum,0.128936,-0.149153
Hotels/Resorts,0.112513,-0.001065
Auto Manufacturing,0.101077,0.450148


In [0]:
# Panda test
base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol='AMZN')
df = pd.read_csv(base_url)
df['symbol'] = 'AMZN'
df.columns
  

Index(['companyName', 'day50MovingAvg', 'day200MovingAvg',
       'month6ChangePercent', 'month3ChangePercent', 'month1ChangePercent',
       'symbol'],
      dtype='object')

In [0]:
get_statistics('AMZN')

,companyName,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent
0,"Amazon.com, Inc.",1.843297,1871.88,1744.88,0.230806,0.066052,0.121094


In [0]:
get_latest_price('AMZN')

,symbol,close
0,AMZN,2009.9


In [0]:
dr.get_data_yahoo('AMZN', date(2019,7,16), date(2019,7,16))


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-07-16,2026.319946,2001.219971,2010.579956,2009.900024,2553900,2009.900024


In [0]:
def get_historical_price(symbol, as_of_date):
  # Leveraging Historical
  import requests
  historical_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/chart/date/{cob}?token=sk_98e397d4bee940488e1f48e9b419508f&chartByDay=true'.format(
                        symbol=symbol, cob=as_of_date.strftime('%Y%m%d'))
  print('Fetching data for {} from:{}'.format(symbol, historical_url))
  return requests.get(historical_url).json()

get_historical_price('AMZN', date(2019,7,16))

Fetching data for AMZN from:https://cloud.iexapis.com/stable/stock/AMZN/chart/date/20190716?token=sk_98e397d4bee940488e1f48e9b419508f&chartByDay=true


[{'change': -11.09,
  'changeOverTime': -0.005487,
  'changePercent': -0.5487,
  'close': 2009.9,
  'date': '2019-07-16',
  'high': 2026.32,
  'label': 'Jul 16, 19',
  'low': 2001.22,
  'open': 2010.58,
  'uClose': 2009.9,
  'uHigh': 2026.32,
  'uLow': 2001.22,
  'uOpen': 2010.58,
  'uVolume': 2618198,
  'volume': 2618198}]

In [0]:
def get_batch_request():
  import requests
  iex_url = 'https://cloud.iexapis.com/stable/stock/aapl/batch?token=sk_98e397d4bee940488e1f48e9b419508f&types=quote,stats&range=1d&last=1'
  return requests.get(iex_url).json()
get_batch_request()

In [0]:
start, end = get_date_ranges()
compute_performance(start, end, 'AMZN')
#get_latest_price('AMZN')

,companyName,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent,Symbol,close
0,"Amazon.com, Inc.",1.707216,1849.96,1774.73,-0.013797,-0.061288,-0.016112,AMZN,1794.16


<h3> Testing Yahoo Historical </h3>

In [0]:
#testing VIX
from scipy.stats import pearsonr
import requests

def get_historical_value(symbol):
  try: 
    data = dr.get_data_yahoo(symbol, date(2018,1,1), date(2019,9,19))[['Adj Close']]
    df =  data.rename(columns={'Adj Close' : symbol})
    return df
  except Exception as e :
    return pd.DataFrame(columns=[symbol])

<h3> Getting VIX values </h3>

In [0]:
#d = {'amzn':  get_historical_value('AMZN')['adj_close'],
#     'aapl' :  get_historical_value('AAPL')['adj_close']}

#for k, v in d.items():
#  print('Correlation beween VIX and {}={}'.format(k, vix_vals.corr(v, method='pearson')))
  
  
  
# https://stackoverflow.com/questions/49350445/correlation-coefficient-of-two-columns-in-pandas-dataframe-with-corr  


<h3>Gettign all stocks from IEX API </h3>

In [0]:
# Get all Stocks from IEX
import requests

def get_all_stocks():
  all_symbols_data = requests.get('https://cloud.iexapis.com/stable/ref-data/iex/symbols?token=sk_98e397d4bee940488e1f48e9b419508f').json()
  return [d['symbol'] for d in all_symbols_data if d['isEnabled']]
  
def get_all_etfs():
  nyse_symbols = requests.get('https://cloud.iexapis.com/stable/ref-data/exchange/nys/symbols?token=sk_98e397d4bee940488e1f48e9b419508f').json()
  nas_symbols = requests.get('https://cloud.iexapis.com/stable/ref-data/exchange/nas/symbols?token=sk_98e397d4bee940488e1f48e9b419508f').json()
  return [d['symbol'] for d in nyse_symbols + nas_symbols if d['type'].lower() == 'et']

def get_all_stocks_data():
  good_ones = get_all_etfs()
  return map(lambda symbol: (symbol, get_historical_value(symbol)), good_ones)


def get_all_exchanges():
  return requests.get('https://cloud.iexapis.com/stable/ref-data/market/us/exchanges?token=sk_98e397d4bee940488e1f48e9b419508f').json()



<p> Testing all stocks in portfolio </p>

In [0]:
portfolio_shares = ['ADAC',
          'AMBS',
          'AMZN',
          'AZFL',
          'ARSC',
          'AAPL',
          'APTY',
          'BTCS',
          'BRK-B',
          'CRNT',
          'CRLBF',
          'XOM',
          'HAON',
          'AGEEF',
          'HMNY',
          'JNJ',
          'LEMIF',
          'NXTTF',
          'NVCN',
          'RNVA',
          'TORC',
          'RTRX',
          'VALE',
          'VZ',
          'DGP',
          'RUSL',
          'REMX',
          'TVIX'
  ]

all_shares = get_all_stocks()

res = map(lambda symbol:(symbol, symbol in all_shares), shares)

invalid = [tpl for tpl in res if not tpl[1]]





In [11]:
# Finding for how many of these we can get data from yahoo
for symbol in shares:
  df = get_historical_value(symbol)
  if df.shape[0] < 2:
    print('No data for:{}'.format(symbol))
  else:
    print('{}={}'.format(symbol, df.shape))

No data for:ADAC
AMBS=(8, 1)
AMZN=(8, 1)
No data for:AZFL
ARSC=(7, 1)
AAPL=(8, 1)
APTY=(8, 1)
BTCS=(8, 1)
BRK-B=(8, 1)
CRNT=(8, 1)
CRLBF=(8, 1)
XOM=(8, 1)
HAON=(7, 1)
AGEEF=(8, 1)
HMNY=(8, 1)
JNJ=(8, 1)
LEMIF=(8, 1)
NXTTF=(8, 1)
NVCN=(8, 1)
RNVA=(7, 1)
TORC=(8, 1)
RTRX=(8, 1)
VALE=(8, 1)
VZ=(8, 1)
DGP=(8, 1)
RUSL=(8, 1)
REMX=(8, 1)
TVIX=(8, 1)


In [0]:
def calculate_correlation_all(vix, all_stocks):
  all_df = [vix]
  res= [vals for _, vals in all_stocks if vals.shape[0] == vix_vals.shape[0]]
  res.append(vix)
  all_data = pd.concat(res, axis=1)
  return all_data.corr('pearson')

def calculate_portfolio_correlation(all_stocks):
  res= [vals for _, vals in all_stocks if vals.shape[0] > 2]
  all_data = pd.concat(res, axis=1)
  return all_data.corr('pearson')


def calculate_correlation(vix, all_stocks):
  result = []
  best = 0
  for symbol, vals in all_stocks:
    if vals.shape[0] == vix.shape[0]:
      concats  = pd.concat([vix, vals], axis = 1)
      corr_matrix = concats.corr(method='pearson')
      corr_with_vix = corr_matrix.loc['^VIX'][1]
      if corr_with_vix > 0 and corr_with_vix > best:
        print('New Corr with {}:{}'.format(symbol, corr_with_vix))
        best = corr_with_vix
  return best

def _get_most_correlated(result_df):
  df = result_df[['^VIX']]
  bad_df = df.index.isin(['^VIX'])
  return df[~bad_df]
  

      
#vix_vals = get_historical_value('^VIX')

all_stocks_data = map(lambda symbol: (symbol, get_historical_value(symbol)), portfolio_shares)
best = calculate_portfolio_correlation(all_stocks_data)      

#



                  

In [23]:
best

,ADAC,AMBS,AMZN,AZFL,ARSC,AAPL,APTY,BTCS,BRK-B,CRNT,CRLBF,XOM,HAON,AGEEF,HMNY,JNJ,LEMIF,NXTTF,NVCN,RNVA,TORC,RTRX,VALE,VZ,DGP,RUSL,REMX,TVIX
ADAC,1.000000,0.828333,-0.618560,0.610857,NaN,-0.410890,0.611412,0.725591,-0.014930,-0.558226,0.075633,-0.051183,0.224831,-0.224707,0.904945,-0.215379,0.715249,0.527293,0.809493,NaN,0.638941,0.109236,-0.175887,-0.579481,0.249146,0.682631,0.789114,0.665474
AMBS,0.828333,1.000000,-0.681729,0.739825,-0.072758,-0.412035,0.766476,0.892113,-0.092584,-0.423667,-0.030493,0.165569,0.285712,-0.301186,0.883814,-0.238401,0.803110,0.662645,0.875303,-0.049153,0.611795,0.296292,-0.040550,-0.649717,0.089650,0.612500,0.865810,0.581721
AMZN,-0.618560,-0.681729,1.000000,-0.678920,-0.368138,0.768550,-0.427063,-0.633879,0.332407,0.187259,0.019456,0.221246,-0.164648,0.157841,-0.713732,0.282708,-0.616212,-0.412092,-0.635175,-0.102236,-0.227157,-0.104741,0.121390,0.516226,-0.196852,-0.309132,-0.647311,-0.776884
AZFL,0.610857,0.739825,-0.678920,1.000000,NaN,-0.384342,0.650421,0.706616,0.060025,-0.376526,-0.081498,0.155747,0.339651,-0.236442,0.772074,0.000618,0.626747,0.678408,0.822311,NaN,0.493336,0.162700,0.020376,-0.391529,0.111123,0.441192,0.632395,0.450102
ARSC,NaN,-0.072758,-0.368138,NaN,1.000000,-0.450961,-0.207348,0.546493,0.072116,0.473544,0.443195,0.367079,NaN,0.473149,0.397768,0.658483,0.553685,0.265415,0.574508,-0.122750,-0.453710,0.559330,0.623943,-0.309035,-0.598554,-0.239442,0.603670,0.432692
AAPL,-0.410890,-0.412035,0.768550,-0.384342,-0.450961,1.000000,-0.069624,-0.331713,0.457365,-0.082263,-0.190100,0.383146,-0.066622,-0.063149,-0.443433,0.308030,-0.310024,-0.086833,-0.336492,0.377739,0.021349,-0.036789,0.185993,0.340911,-0.168509,-0.149995,-0.387646,-0.661239
APTY,0.611412,0.766476,-0.427063,0.650421,-0.207348,-0.069624,1.000000,0.846284,0.038872,-0.339831,-0.385622,0.317408,0.390047,-0.681747,0.663793,-0.201784,0.788371,0.855239,0.719722,0.075916,0.658326,0.601593,0.157973,-0.661819,-0.255131,0.275385,0.762130,0.456076
BTCS,0.725591,0.892113,-0.633879,0.706616,0.546493,-0.331713,0.846284,1.000000,-0.187308,-0.397206,-0.212532,0.254382,0.274878,-0.512751,0.793527,-0.364371,0.919776,0.726032,0.780121,-0.199963,0.520880,0.509384,0.067212,-0.788046,-0.071631,0.454537,0.933930,0.607062
BRK-B,-0.014930,-0.092584,0.332407,0.060025,0.072116,0.457365,0.038872,-0.187308,1.000000,-0.001367,0.083315,0.383548,0.114033,0.139935,0.019744,0.744056,-0.227410,0.195009,0.093637,0.161151,0.274882,-0.072026,0.297831,0.414513,-0.193320,0.113309,-0.178938,-0.367547
CRNT,-0.558226,-0.423667,0.187259,-0.376526,0.473544,-0.082263,-0.339831,-0.397206,-0.001367,1.000000,-0.098032,0.126301,0.071064,0.105536,-0.578542,0.120588,-0.474276,-0.238307,-0.519230,0.085423,-0.281414,0.242340,0.273805,0.251865,-0.490573,-0.630664,-0.396400,-0.181604


In [26]:
best[(best > 0.6).all(1)]

,ADAC,AMBS,AMZN,AZFL,ARSC,AAPL,APTY,BTCS,BRK-B,CRNT,CRLBF,XOM,HAON,AGEEF,HMNY,JNJ,LEMIF,NXTTF,NVCN,RNVA,TORC,RTRX,VALE,VZ,DGP,RUSL,REMX,TVIX


In [0]:
res = _get_most_correlated(best)
sorted_df = res.sort_values('^VIX', ascending=False)
sorted_df.head(10)


,^VIX
VIIX,0.698999
BIS,0.620278
TVIX,0.529566
ZBIO,0.528948
SQQQ,0.407953
FBZ,0.120247
EWZS,0.106043
ISHG,0.065553
DSLV,0.065009
TUR,0.059713


In [0]:
idx = sorted_df.iloc[0]

print(idx.values.tolist()[0])
print(sorted_df.index[0])

0.07980534120952935
AAME


In [0]:
sorted_df.index


Index(['AAME', 'AAU', 'ABBV', 'ABEV', 'AAP', 'AAL', 'ABDC', 'ABIL', 'AAMC',
       'AAT', 'AAN', 'AAC', 'ABC', 'AA', 'ABEO', 'AAON', 'ABCB', 'ABG', 'A',
       'AAOI', 'AADR', 'AB', 'AAWW', 'ABB', 'AAXN', 'AAPL', 'AAXJ', 'AABA'],
      dtype='object')

In [0]:
corr_matrix = all_df.corr(method='pearson')

In [0]:
corr_matrix

<h3> Getting list of all ETFS </h3>



In [0]:
xl.shape
xl.columns
  
  
## Spark
"""
val firstRDD: RDD[Double] = yourDF.select("field1").map(row => row.getDouble(0))
val secondRDD: RDD[Double] = yourDF.select("field2").map(row => row.getDouble(0))
val corr = Statistics.corr(firstRDD, secondRDD, "spearman")


"""
  
  




Index(['S.No.', 'Name', 'Ticker'], dtype='object')

In [0]:
len(res)

NameError: ignored